# Collision Avoidance - Live Demo
In this notebook we'll use the model we trained to detect whether the robot is free or blocked to enable a collision avoidance behavior on the robot. It is assumed that the optimized neural network has already been created and loaded into the Jupyter Lab file browser.

## TensorRT

In [ ]:
import cv2
import torch
from torchvision.models import resnet18
from torch2trt import TRTModule
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
from utils import preprocess
import torch.nn.functional as F
import numpy as np
import time

Load the optimized model by executing the cell below.

In [ ]:
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_model_resnet18_trt.pth'))

Inizialize `car` and `camera` objects.

In [ ]:
car = NvidiaRacecar()
camera = CSICamera(width=224, height=224, capture_fps=65)

## Live Demo

Due to the JetRacer's limited steering angle and obstacle visibility during the steering process, a specific time interval is required for successful obstacle avoidance. In the suggested demo, the robot will steer for 2 seconds upon obstacle detection. Nevertheless, this value should be dynamically modified to align with specific operational needs.

In [ ]:
prob_blocked = 0
threshold = 0.5

turning = False  # Flag to indicate if the car is turning
turn_time = time.time()  # Start a timer

while True:
    image = camera.read()
    image_proc = preprocess(image).half()
    output = model_trt(image_proc)
    y = F.softmax(output, dim=1)
    prob_blocked = float(y.flatten()[0])

    if prob_blocked < threshold and not turning:
        car.steering = 0.0
    else:
        if not turning:
            car.steering = -0.8  # Choose a steering angle
            turning = True  # Set the turning flag
            turn_time = time.time()  # Start a timer

        if time.time() - turn_time >= 2 and prob_blocked < threshold:
            car.steering = 0.0  # Straighten the wheels
            turning = False  # Clear the turning flag

    car.throttle = 0.18  # Choose car.throttle